https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html

https://nikolasent.github.io/computervision/opencv/calibration/2024/12/20/Practical-OpenCV-Refinement-Techniques.html

In [22]:
import glob as glob
import os

import numpy as np

import calibrationlib

# This calibration operation has to be done once theoritically, if the cameras are securely fixed together.
# A minimal set of ~10 images is required officially ; in practice we take around 50 images, trying to cover 
# the corners of the image, different orientations of the chessboard and a range of distances to the sensor
# consistent with our applications.
# The more images are well-exposed (taken outside for example), the less noise you will have and the better
# the chessboard detection will work

path_calibration = (r"./data/calibration/classical_system")

# Calibration paths for left ('Camera1') and right ('Camera2') chessboard images
path_images = [
    path_calibration + "/Left",
    path_calibration + "/Right",
]

image_type = "jpg"

# Chessboard pattern size (width, height)
chessboard_pattern = (13, 8)
# Chessboard pattern cell size (here in mm) : used for scaling calibration parameters
chessboard_cell_size = 40

calibration = calibrationlib.CameraCalibration(chessboard_pattern, chessboard_cell_size)

count = 0

object_points_cam1 = []
image_points_cam1 = []
object_points_cam2 = []
image_points_cam2 = []

success_images_cam1 = []
success_images_cam2 = []

name = path_calibration + "/errors.log"
log = open(name, "w")

In [23]:
# Computation of the calibration parameters for both left and right cameras (mandatory), then for the stereo camera pair

# Left/first camera
log_line = "Calibration - Left/first camera\r"
log.write(log_line)
print(log_line)

ret1 = False
if not (os.path.exists(path_images[0] + "/image_points.npy")):
    (
        ret1,
        reprojection_error_cam1,
        success_images_cam1,
        roi_cam1,
        camera_matrix_cam1,
        dist_coefs_cam1,
        r_vecs_cam1,
        t_vecs_cam1,
        object_points_cam1,
        image_points_cam1,
        image_size_cam1,
    ) = calibration.single_calibration(
        path_images[0],
        image_type,
        log_file=log,
    )

    tmp = np.array(object_points_cam1)
    name = path_images[0] + "/object_points"
    np.save(name, tmp)
    tmp = np.array(image_points_cam1)
    name = path_images[0] + "/image_points"
    np.save(name, tmp)
    tmp = np.array(success_images_cam1)
    name = path_images[0] + "/success_images"
    np.save(name, tmp)

    tmp = np.array(image_size_cam1)
    name = path_images[0] + "/image_size"
    np.save(name, tmp)

    tmp = np.array(camera_matrix_cam1)
    name = path_images[0] + "/camera_matrix"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1)
    name = path_images[0] + "/dist_coefs"
    np.save(name, tmp)
    tmp = np.array(r_vecs_cam1)
    name = path_images[0] + "/r_vecs"
    np.save(name, tmp)
    tmp = np.array(t_vecs_cam1)
    name = path_images[0] + "/t_vecs"
    np.save(name, tmp)
    tmp = np.array(reprojection_error_cam1)
    name = path_images[0] + "/reprojection_error"
    np.save(name, tmp)
else:
    camera_matrix_cam1 = np.load(path_images[0] + "/camera_matrix.npy")
    dist_coefs_cam1 = np.load(path_images[0] + "/dist_coefs.npy")
    image_size_cam1 = np.load(path_images[0] + "/image_size.npy")
    object_points_cam1 = np.load(path_images[0] + "/object_points.npy")
    object_points_cam1 = list(object_points_cam1)
    image_points_cam1 = np.load(path_images[0] + "/image_points.npy")
    image_points_cam1 = list(image_points_cam1)
    success_images_cam1 = np.load(path_images[0] + "/success_images.npy")
    success_images_cam1 = list(success_images_cam1)

    ret1 = True

log_line = f"End of left/first camera calibration successful : {ret1}\r"
log.write(log_line)
print(log_line)

Calibration - Left/first camera
End of left/first camera calibration successful : True


In [24]:
camera_matrix_cam1

array([[3.43030370e+03, 0.00000000e+00, 2.36490980e+03],
       [0.00000000e+00, 3.43008682e+03, 1.62809585e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [25]:
dist_coefs_cam1

array([[ 0.01741913],
       [-0.00377566],
       [ 0.00063558],
       [-0.00152854],
       [-0.01996182]])

In [26]:
# Right/second camera
log_line = "Calibration - Right/second camera\r"
log.write(log_line)
print(log_line)

ret2 = False
if not (os.path.exists(path_images[1] + "/image_points.npy")):
    (
        ret2,
        reprojection_error_cam2,
        success_images_cam2,
        roi_cam2,
        camera_matrix_cam2,
        dist_coefs_cam2,
        r_vecs_cam2,
        t_vecs_cam2,
        object_points_cam2,
        image_points_cam2,
        img_size_cam2,
    ) = calibration.single_calibration(
        path_images[1],
        image_type,
        log_file=log,
    )

    tmp = np.array(object_points_cam2)
    name = path_images[1] + "/object_points"
    np.save(name, tmp)
    tmp = np.array(image_points_cam2)
    name = path_images[1] + "/image_points"
    np.save(name, tmp)
    tmp = np.array(success_images_cam2)
    name = path_images[1] + "/success_images"
    np.save(name, tmp)

    tmp = np.array(image_size_cam1)
    name = path_images[1] + "/image_size"
    np.save(name, tmp)
    
    tmp = np.array(camera_matrix_cam2)
    name = path_images[1] + "/camera_matrix"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2)
    name = path_images[1] + "/dist_coefs"
    np.save(name, tmp)
    tmp = np.array(r_vecs_cam2)
    name = path_images[1] + "/r_vecs"
    np.save(name, tmp)
    tmp = np.array(t_vecs_cam2)
    name = path_images[1] + "/t_vecs"
    np.save(name, tmp)
    tmp = np.array(reprojection_error_cam2)
    name = path_images[1] + "/reprojection_error"
    np.save(name, tmp)
else:
    camera_matrix_cam2 = np.load(path_images[1] + "/camera_matrix.npy")
    dist_coefs_cam2 = np.load(path_images[1] + "/dist_coefs.npy")
    image_size_cam1 = np.load(path_images[0] + "/image_size.npy")
    object_points_cam2 = np.load(path_images[1] + "/object_points.npy")
    object_points_cam2 = list(object_points_cam2)
    image_points_cam2 = np.load(path_images[1] + "/image_points.npy")
    image_points_cam2 = list(image_points_cam2)
    success_images_cam2 = np.load(path_images[1] + "/success_images.npy")
    success_images_cam2 = list(success_images_cam2)
    
    ret2 = True

log_line = f"End of right/second camera calibration successfully achieved : {ret2}\r"
log.write(log_line)
print(log_line)

Calibration - Right/second camera
End of right/second camera calibration successfully achieved : True


In [27]:
camera_matrix_cam2

array([[3.41779178e+03, 0.00000000e+00, 2.37339789e+03],
       [0.00000000e+00, 3.41827258e+03, 1.61994480e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [28]:
dist_coefs_cam2

array([[ 0.02194274],
       [-0.0166134 ],
       [ 0.00044922],
       [-0.00137957],
       [ 0.00328596]])

In [29]:
# Calibration of the stereo pair
log_line = "Stereo calibration between left/first and right/second cameras\r"
log.write(log_line)
print(log_line)

if (ret1 is True) and (ret2 is True):

    # We keep only image pairs on which were successfully detected the chessboard corners
    # for both cameras, and store relative objects and image points for further stereo
    # calibration
    shortest_list_cam1 = success_images_cam1
    longest_list_cam1 = success_images_cam2
    if len(success_images_cam1) > len(success_images_cam2):
        shortest_list_cam1 = success_images_cam2
        longest_list_cam1 = success_images_cam1

    index_points_cam1 = []
    index_points_cam2 = []
    for el in shortest_list_cam1:
        if el in longest_list_cam1:
            index_points_cam1.append(success_images_cam1.index(el))
            index_points_cam2.append(success_images_cam2.index(el))

    log_line = f"Common images : {(np.array(success_images_cam1))[index_points_cam1]}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 1 : {index_points_cam1}\r"
    log.write(log_line)
    log_line = f"Images indexes for camera 2 : {index_points_cam2}\r"
    log.write(log_line)

    # Resulting list of common points for the two cameras (i.e. from the image pairs on which
    # the chessboard has been successfully deteted in both cameras
    filtered_object_points_cam1 = np.array(object_points_cam1)
    filtered_object_points_cam1 = filtered_object_points_cam1[index_points_cam1]
    filtered_object_points_cam1 = list(filtered_object_points_cam1)
    
    filtered_image_points_cam1 = np.array(image_points_cam1)
    filtered_image_points_cam1 = filtered_image_points_cam1[index_points_cam1]
    filtered_image_points_cam1 = list(filtered_image_points_cam1)
    
    filtered_image_points_cam2 = np.array(image_points_cam2)
    filtered_image_points_cam2 = filtered_image_points_cam2[index_points_cam2]
    filtered_image_points_cam2 = list(filtered_image_points_cam2)

    # We compute now stereo calibration : camera_matrices and distorsion coefficients for each
    # camera may be slightly modified (depending on the flags parameters : in our case, the output 
    # values are strictly the same as the input ones), and we get the rotation matrix R and 
    # translation matrix T that bring points in the left camera coordinate system to points in the 
    # right camera coordinate system (you may read the doc : documentation/OpenCV - stereoCalibrate) 
    (
        ret,
        camera_matrix_cam1_stereo,
        dist_coefs_cam1_stereo,
        camera_matrix_cam2_stereo,
        dist_coefs_cam2_stereo,
        R,
        T,
    ) = calibration.stereo_calibration(
        filtered_object_points_cam1,
        filtered_image_points_cam1,
        filtered_image_points_cam2,
        camera_matrix_cam1,
        dist_coefs_cam1,
        camera_matrix_cam2,
        dist_coefs_cam2,
        image_size_cam1,
        log_file=log,
    )

    # We save the useful parameters, from which further rectification of images will 
    # be possible for matching. The rectification maps will be loaded once at the 
    # beginning of the matching code, and applied to each images pair.
    tmp = np.array(camera_matrix_cam1_stereo)
    name = path_images[0] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam1_stereo)
    name = path_images[0] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(camera_matrix_cam2_stereo)
    name = path_images[1] + "/camera_matrix_from_stereo"
    np.save(name, tmp)
    tmp = np.array(dist_coefs_cam2_stereo)
    name = path_images[1] + "/dist_coefs_from_stereo"
    np.save(name, tmp)
    tmp = np.array(R)
    name = path_calibration + "/R"
    np.save(name, tmp)
    tmp = np.array(T)
    name = path_calibration + "/T"
    np.save(name, tmp)
    tmp = np.array(ret)
    name = path_calibration + "/reprojection_error"
    np.save(name, tmp)

    log_line = "End of stereo calibration successfully achieved\r"
    log.write(log_line)
    print(log_line)
else:
    log_line = "Problem with the calibration of one of the two cameras - no way to perform stereo calibration"
    log.write(log_line)
    print(log_line)

log_line = "End of calibration process"
log.write(log_line)
print(log_line)

Stereo calibration between left/first and right/second cameras
End of stereo calibration successfully achieved
End of calibration process


In [30]:
camera_matrix_cam1

array([[3.43030370e+03, 0.00000000e+00, 2.36490980e+03],
       [0.00000000e+00, 3.43008682e+03, 1.62809585e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [31]:
camera_matrix_cam1_stereo

array([[3.43030370e+03, 0.00000000e+00, 2.36490980e+03],
       [0.00000000e+00, 3.43008682e+03, 1.62809585e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [32]:
dist_coefs_cam1

array([[ 0.01741913],
       [-0.00377566],
       [ 0.00063558],
       [-0.00152854],
       [-0.01996182]])

In [33]:
dist_coefs_cam1_stereo

array([[ 0.01741913],
       [-0.00377566],
       [ 0.00063558],
       [-0.00152854],
       [-0.01996182]])

In [34]:
camera_matrix_cam2

array([[3.41779178e+03, 0.00000000e+00, 2.37339789e+03],
       [0.00000000e+00, 3.41827258e+03, 1.61994480e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [35]:
camera_matrix_cam2_stereo

array([[3.41779178e+03, 0.00000000e+00, 2.37339789e+03],
       [0.00000000e+00, 3.41827258e+03, 1.61994480e+03],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [36]:
dist_coefs_cam2

array([[ 0.02194274],
       [-0.0166134 ],
       [ 0.00044922],
       [-0.00137957],
       [ 0.00328596]])

In [37]:
dist_coefs_cam2_stereo

array([[ 0.02194274],
       [-0.0166134 ],
       [ 0.00044922],
       [-0.00137957],
       [ 0.00328596]])

In [38]:
R

array([[ 9.99980105e-01, -2.44344321e-05, -6.30780411e-03],
       [ 5.57914454e-05,  9.99987643e-01,  4.97101761e-03],
       [ 6.30760470e-03, -4.97127063e-03,  9.99967750e-01]])

In [39]:
T

array([[-6.47725249e+01],
       [-6.19450306e-02],
       [ 2.04588885e-01]])